<a href="https://colab.research.google.com/github/r-chaudhary/Artificial-Intelligent-Repo/blob/master/AI_Astar_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ai Practical
# A* Search

## Graph Structure

In [482]:
class Node:
    def __init__(self, name, value=None, state=None, coords=None):
        self.name = name
        self.value = value
        self.state = state
        self.coords = coords
        self.edges = {}

    def __str__(self):
        return self.name

    def addEdge(self, node, cost=1):
        self.edges[node]=  {"cost":cost}

class Graph:
    def __init__(self):
        self.nodes = {}
        self.graph = dict()

    def __len__(self):
        return len(self.nodes)

    def addNode(self, name, value=None, state=None, coords=None):
        # print("Node : ",name, value, state, coords)
        node = Node( name, value, state, coords)
        self.nodes[name] = node

    def addEdge(self,from_node, to_node, cost=1):
        self.nodes[from_node].addEdge(to_node,cost)

## Algorithm

In [483]:
# A* Search Algorithm 

def A_star(graph, start, goal, heuristics):

    # function of return most nearest cell 
    def piority_node(node_list):
        fscore = node_list[0].fscore
        priority = node_list[0]

        for node in node_list:
            if fscore > node.fscore:
                priority = node
        # f_score = [{i.fscore:i} for i in node_list]
        return priority

    # function of return all adjacent nodes
    def neighbour(node):
        neighbour_list = []
        for i in node.edges.keys():
            if graph.nodes[i].state == "free" or graph.nodes[i].state == "goal" :
                neighbour_list.append(graph.nodes[i])
        return neighbour_list 

    # function of return shortest
    def construct_path(node):
        path = []
        state = "goal"
        current_node = node
        while True:
            parent = graph.nodes[str(current_node.parent)]
            state = parent.state
            if state == "start":
                break
            parent.state = "path"
            path.append(str(parent))
            current_node = parent
        return path
            
    queue = []                     # Priority Queue
    path = []               # Path
    discovered = [str(start),]     # dicovered nodes

    graph.nodes[start].fscore = 0       # setting start 
    graph.nodes[start].cost = 0         #
    graph.nodes[start].parent =  start  # 
    graph.nodes[start].heuristics  = heuristics(graph.nodes[start])
    queue.append(graph.nodes[start])
    success = False
    while queue:                    # Till queue is not empty
        # Step 1 : Find the nearest node
        current_node = piority_node(queue)
        queue.remove(current_node)

        # Step 2 : Add current node to path
        path.append(current_node)

        # Step 3 : If current node is not goal and search for goal
        #          Find neighbour current node
        neighbour_list = neighbour(current_node)


        # Step 4 : Find the nearest node to the goal in neighbours

        for node in neighbour_list:

            # If neighbour node is already discovered skip
            if str(node) in discovered:
                continue

            # Add neighbour node to discoverd
            discovered.append(str(node))

            # Set parent of the neighbour node to current node
            node.parent = str(current_node)

            # # If goal found
            if str(node) == goal:
                node.cost = current_node.cost + current_node.edges[str(node)]["cost"]
                node.heuristics = heuristics(node)
                node.fscore = node.cost + node.heuristics
                node.state = "goal"
                print("Path found after steps :",node.cost)
                path.append(node)
                success = True
                return construct_path(node)
            
            # Calculate cost heuristics fscore
            node.cost = current_node.cost + current_node.edges[str(node)]["cost"]
            node.heuristics = heuristics(node)
            node.fscore = node.cost + node.heuristics

            # Condition for valid nodes to path
            if current_node.heuristics < node.heuristics + node.cost:

                # Add neighbour node to path
                node.state="traversed"

                # Add neighbours to queue
                queue.append(node)

    print("No Path Found")



## Problem

In [484]:
import numpy as np
import sys
class Board:
    def __init__(self, start=None, goal=None, height=11, width=11):
        # Parameters
        self.height = height
        self.width = width

        # Points
        self.start_coord = start
        self.goal_coord = goal
        if self.start_coord == None:
            self.start_coord = (height-1,0)
        if self.goal_coord == None:
            self.goal_coord = (0,width-1)

        self.start = None
        self.goal = None

        # board Board Graph 
        self.graph = self.draw_board()

        # Paths
        self.path = None
        self.traversed_path = None

        # Draw Board
        # self.draw()

    def draw_board(self):
        graph = Graph()

        # Adding all nodes
        id = 0
        for x in range(self.height):
            for y in range(self.width):
                graph.addNode(str(id),state="free",coords=(x,y))
                id += 1

        # Blockes detail
        def blocks(x,y,height):
            block = []
            for i in range(height):
                b = (x+i,y)
                block.append(b)
            return block


        blockes = []
        def draw_blocks():
            mid_point = (self.height/2, self.width/2)
            half_mid_point = (mid_point[0]/2, mid_point[1]/2)
            x = mid_point[0] - half_mid_point[0]
            y = mid_point[1]
            block = blocks(0,int(y),int(mid_point[0]))
            return block

        blockes.extend(draw_blocks())
        # Connecting all nodes with edges
        for node in graph.nodes.values():
            # print(node.coords)
            x = node.coords[0]
            y = node.coords[1]

            edge_list = [ (x-1,y), (x,y-1), (x,y+1), (x+1,y), (x-1,y-1), (x+1,y+1), (x-1,y+1), (x+1,y-1),]
            # print("EDGE LIST : ",edge_list)
            graph_add=[]
            for i in edge_list:
                if i[0] < 0 or i[0] > self.height: # Skipping Invalid connections
                    continue
                elif i[1] < 0 or i[1] > self.width: # Skipping Invalid connections
                    continue
                elif i in graph_add:
                    continue
                else:
                    for value in graph.nodes.values():
                        if value.coords == i:
                            edge = str(value)
                            break
                    # print("EDGE : ",i,edge)
                    graph_add.append(edge)
                    node.addEdge(edge)
            # print("GRAPH LIST : ",graph_add)
                    

            # Adding Blockes
            for block in blockes:
                if block == node.coords:
                    node.state = "blocked"

            if self.start_coord == node.coords:
                node.state = "start"
                self.start = node.name

            if self.goal_coord == node.coords:
                node.state = "goal"
                self.goal = node.name

        return graph

    def draw(self, result=True):
        nodes = np.empty(shape=(self.height,self.width), dtype="O")
        for node in self.graph.nodes.values():
            coords = node.coords
            nodes[coords[0]][coords[1]] = node.state

        print("\u2587 {:<15} \t \033[{}m\u2587\033[00m {:<15}".format("Free Cell",33,"Blocked Cell"))
        print("\033[{}m\u2587\033[00m {:<15} \t \033[{}m\u2587\033[00m {:<15}".format(34,"Path",90,"Traversed "))
        print("\033[{}m\u2587\033[00m {:<15} \t \033[{}m\u2587\033[00m {:<15}".format(32,"Start Cell",31,"Goal Cell"))
        print()
        for x in range(len(nodes)):
            for y in range(len(nodes[x])):  
                if nodes[x][y] == "free":                # Free Cells
                        print("\u2587",end=" ")
                elif nodes[x][y] == "blocked":              # Blocked Cells
                    print("\033[33m\u2587\033[00m",end=" ")
                elif nodes[x][y] == "path":              # Shortest Path Cells
                    print("\033[34m\u2587\033[00m",end=" ")
                elif nodes[x][y] == "traversed":              # All Path travesed Cells
                    print("\033[90m\u2587\033[00m",end=" ")
                elif nodes[x][y] == "start":              # Start Cell
                    print("\033[32m\u2587\033[00m",end=" ")
                elif nodes[x][y] == "goal":              # End Cell
                    print("\033[31m\u2587\033[00m",end=" ")
            print()



## Solution

In [485]:
# Create a Board ... Simple board
board = Board(height = 20, width = 30)

In [486]:
 def heuristics(board):
    # Coordiantes of goal cell 
    coord = np.array([board.goal_coord[0],board.goal_coord[1]])

    # Calculation of heuristics
    def generate_heuristics(node):
        # Coordinates of goal cell subtrated by current cell
        f_score = coord - node.coords
        return max(abs(f_score[0]),abs(f_score[1]))
    return generate_heuristics

In [487]:
# Running A star Search
path = A_star(board.graph, board.start, board.goal, heuristics(board))
board.draw()

Path found after steps : 30
▇ Free Cell       	 ▇ Blocked Cell   
▇ Path            	 ▇ Traversed      
▇ Start Cell      	 ▇ Goal Cell      

▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ ▇ 
▇ ▇